In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn import tree
import matplotlib.pyplot as plt

In [2]:
def get_data(path: str) -> pd.DataFrame:
    """
    Getting data from local file
    """
    file_path = Path().parent / path

    data = pd.read_csv(file_path)

    return data

In [3]:
def preprocess_data(data: pd.DataFrame, nan_columns: list[str]) -> pd.DataFrame:
    """
    Dropping nan values, filling dataframe, encoding features
    """
    data = data.drop(columns=["Unnamed: 0"])

    data = data.dropna(subset=nan_columns)

    data = pd.get_dummies(data, columns=["ethnicity"], drop_first=True)

    label_encoder = LabelEncoder()
    data["gender_binary"] = label_encoder.fit_transform(data["gender"])

    return data

In [4]:
def get_model(X_train: pd.DataFrame, y_train: pd.DataFrame, depth: int) -> DecisionTreeClassifier:
    """
    Decision Tree was chosen as a predictinve model since we can easier analyze features that actually affects the target graphically
    """
    decision_tree = DecisionTreeClassifier(max_depth=depth)
    decision_tree.fit(X_train, y_train)

    return decision_tree

In [5]:
def plot_tree(model: DecisionTreeClassifier, features: list[str], depth: int) -> None:
    """
    120 70 is most optimal for such depth (7)
    """
    plt.figure(figsize=(120, 70))  
    tree.plot_tree(
        decision_tree=model, 
        feature_names=features, 
        filled=True, 
        precision=2, 
        max_depth=depth,
        fontsize=10 
    )
    plt.show()

In [6]:
data = get_data(path="data/sample_diabetes_mellitus_data.csv") # getting data

data = preprocess_data(data=data, nan_columns=["age", "gender", "ethnicity"]) # specifying nan calumns

In [7]:
# specifying features and applyung them to get X

features = [
    "age", 
    "height", 
    "bmi", 
    "aids", 
    "cirrhosis", 
    "hepatic_failure", 
    "immunosuppression", 
    "leukemia", 
    "lymphoma", 
    "solid_tumor_with_metastasis"
]

X = data[features]

y = data["diabetes_mellitus"]

In [8]:
# Splitting data, specifying depth (how deep the decision tree modes can go)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

depth = 7

model = get_model(X_train, y_train, depth=depth)

In [9]:
train_predictions = model.predict_proba(X_train)[:, 1]
test_predictions = model.predict_proba(X_test)[:, 1]

data.loc[X_train.index, 'train_predictions_dt'] = train_predictions
data.loc[X_test.index, 'test_predictions_dt'] = test_predictions

train_roc_auc_dt = roc_auc_score(y_train, train_predictions)
test_roc_auc_dt = roc_auc_score(y_test, test_predictions)

In [10]:
plot_tree(model=model, features=features, depth=depth)

print(f'Train: {train_roc_auc_dt}, Test: {test_roc_auc_dt}')

Train: 0.7406010185681461, Test: 0.6531285433037552
